# Hilfsfunktionen
Im folgenden sollen Funktionen implementiert werden, die an verschiedenen Stellen in der Entwicklung des Mühlespiels genutzt werden können. Dazu sind sie so einfach wie möglich gehalten.

`count_player_pieces()` zählt die Steine, die ein anzugebender Spieler gerade auf dem Spielfeld hat.

In [ ]:
def count_player_pieces(state, player):
    [_, board] = state
    return [position for ring in board for position in ring].count(player)

`empty_positions()` gibt eine Liste mit Tupeln zurück. Jedes Tupel beschreibt eine Stelle auf dem Spielfeld, auf der kein Spielstein steht. So erhält man alle leeren Positionen.

In [ ]:
def empty_positions(state):
    [_, board] = state
    return [(ring, cell) for ring in range(3)
                        for cell in range(8)
                        if  board[ring][cell] == 0 
           ]

`neighboring_positions()` liefert für eine gegebene Position auf dem Spielfeld alle benachbarten Felder. Die Position wird dabei in der der Form `(ring, cell)` übergeben. Je nachdem wo sich die übergebene Stelle befindet, hat sie 2, 3 oder 4 Nachbarfelder, die es zu ermitteln gilt.

In [ ]:
def neighboring_positions(position):
    ring, cell = position
    
    left_neighbor  = (ring, (cell - 1) % 8)
    right_neighbor = (ring, (cell + 1) % 8)
        
    positions = [left_neighbor, right_neighbor]
        
    if cell in (1, 3, 5, 7):
        if ring in (0, 2) :
            positions.append((1, cell))
        else:
            positions.append((ring - 1, cell))
            positions.append((ring + 1, cell))
    return positions

`player_pieces()` gibt eine Liste mit allen Positionen eines zu übergebenden Spielers zurück.

In [ ]:
def player_pieces(state, player):
    [_, board] = state
    return [(ring, cell) for ring in range(3)
                        for cell in range(8)
                        if  board[ring][cell] == player
           ]

`opponent()` gibt den Gegner des übergebenen Spielers zurück.

In [ ]:
def opponent(player):
    return 3 - player

`is_movable()` gibt für einen übergebenen Stein an, ob dieser noch verschiebbar ist.

In [ ]:
def is_movable(state, position):
    neighboring = set(neighboring_positions(position))
    for (ring, cell) in neighboring:
        if state[1][ring][cell] == 0:
            return True
    return False

## Mühlen
`find_mills()` gibt alle Mühlen zurück, die ein Spieler auf dem übergebenen Spielbrett hat
Dazu werden zunächst alle Mühlen identifiziert, die auf dem Spielfeld zu finden sind. Hierzu werden in der ersten Schleife alle Mühlen ermittelt, die sich auf den Ringen befinden. Die zweite Schleife erkennt alle Mühlen, die sich vertikal über alle Ringe erstrecken. Mühlen werden dabei als Tupel der Form ***(Spieler, Ring, Zelle)*** gespeichert. Die Zelle ist in diesem Fall die Zelle einer Mühle mit dem niedrigsten Index. Ein Unterschied besteht bei diesem Format bei den Mühlen, die vertikal verlaufen. Für diese wird der äußerste Ring mit dem Index 0 eingetragen.

In [ ]:
def find_mills(board, player):
    mills = set()
    
    for ring in range (0,3):
        for cell in [0,2,4,6]:
                if(board[ring][cell] == board[ring][cell+1] == board[ring][(cell+2) % 8] == player):
                    mills.add((player, ring, cell))

    for cell in [1,3,5,7]:
        if(board[0][cell] == board[1][cell] == board[2][cell] == player):
            mills.add((player, 0, cell))

            
    # TODO: Später ersetzen durch:
    #for ring in range (0,3):
    #    for cell in [0,2,4,6]:
    #            if(state[1][ring][cell] == state[1][ring][cell+1] == state[1][ring][(cell+2) % 8] == player):
    #                opponent_mill_stones.add((ring,cell))
    #                opponent_mill_stones.add((ring,cell+1))
    #                opponent_mill_stones.add((ring,(cell+2) % 8))
    #for cell in [1,3,5,7]:
    #    if(state[1][0][cell] == state[1][1][cell] == state[1][2][cell] == player):
    #        opponent_mill_stones.add((0,cell))
    #        opponent_mill_stones.add((1,cell))
    #        opponent_mill_stones.add((2,cell))
            
    return mills

`count_new_mills()` nimmt zwei Mengen von Mühlen und berechnet die Anzahl der Unterschiede zwischen `old_mills` und `new_mills`.

In [ ]:
def count_new_mills(old_mills, new_mills):
    return len(new_mills.difference(old_mills))

Die Funktion `handle_mills()` gibt die Anzahl der im letzten Zug erzeugten Mühlen für den aktuellen Spieler zurück. Hierzu darf der Spieler erst nach dem Ausführen dieser Funktion wechseln.

Im ersten Abschnitt "find all current mills on the board" werden zunächst alle Mühlen identifiziert, die auf dem Spielfeld zu finden sind. Hierzu werden in der ersten Schleife alle Mühlen ermittelt, die sich auf den Ringen befinden. Die zweite Schleife erkennt alle Mühlen, die sich vertikal über alle Ringe erstrecken. Mühlen werden dabei als Tupel der Form ***(Spieler, Ring, Zelle)*** gespeichert. Die Zelle ist in diesem Fall die Zelle einer Mühle mit dem niedrigsten Index. Ein Unterschied besteht bei diesem Format bei den Mühlen, die vertikal verlaufen. Für diese wird der äußerste Ring mit dem Index 0 eingetragen.

Da die Mühlen als Menge gespeichert werden ist es einfach möglich zu bestimmen, ob neue Mühlen entstanden sind. Wurden neue Mühlen erkannt werden durch die Differenz der bestehenden Mühlen zu den existierenden Mühlen die Anzahl der neuen Mühlen bestimmt. Diese Anzahl wird zurückgegeben und die Menge der Mühlen in der Game-Klassen-Instanz aktualisiert.


In [ ]:
#TODO: Später löschen
def handle_mills(game):
    state = game.state
    player = game.current_player
    old_mills = game.mills
    new_mills = set()
    current_player_mills = set()

    #find all current mills on the board
    for ring in range (0,3):
        for cell in [0,2,4,6]:
            if(state[1][ring][cell] == state[1][ring][cell + 1] == state[1][ring][(cell + 2) % 8] != 0):
                new_mills.add((state[1][ring][cell], ring ,cell))

    for cell in [1,3,5,7]:
        if(state[1][0][cell] == state[1][1][cell] == state[1][2][cell] != 0):
            new_mills.add((state[1][0][cell], 0 ,cell))

    #set new mills
    game.mills = new_mills   
            
    #if the current mills are the same mills as the old ones or there are now less mills on the board => no new mills
    if(old_mills == new_mills or len(old_mills) > len(new_mills)):
        return(0)

    new_mills = new_mills.difference(old_mills) #new_mills are now really the new Mills

    #pics mills that belong to the current player 
    for (mill_owner, ring, cell) in new_mills:
        if mill_owner == player:
            current_player_mills.add((mill_owner, ring, cell))

    return len(current_player_mills) #returns count of new mills for the current player

`get_opponent_beatable_stones()` gibt eine Liste mit entfernbaren Steinen des Gegners zurück. Dies ist notwendig, wenn auf Grund des Bildens einer Mühle Steine des Gegners entfernt werden dürfen. Laut den Regeln dürfen keine Steine aus Mühlen entfernt werden, es sei denn, alle Steine des Gegners sind in Mühlen. 
Dazu werden alle Mühlsteine des Gegners in einer Menge gespeichert. Anschließend wird geprüft, welche Steine nicht in einer Mühle sind und diese in einer Liste zurückgegeben. Ist die Liste jedoch leer, werden alle Steine des Gegners zurückgegeben.

In [ ]:
def get_opponent_beatable_stones(state, current_player):
    opponent_player = opponent(current_player)
    opponent_mill_stones = set()

    #find all opponent_mill_stones | wenn ein Stein in einer Mühle ist, darf er nicht entfernt werden
    _,board = state

    #TODO: später ersetzen durch
    #opponent_mill_stones = [[ring, cell] for ring, cell in mill for mill in find_mills(state, opponent_player)]
    for ring in range (0, 3):
        for cell in [0, 2, 4, 6]:
                if(board[ring][cell] == board[ring][cell+1] == board[ring][(cell+2) % 8] == opponent_player):
                    opponent_mill_stones.add((ring, cell))
                    opponent_mill_stones.add((ring, cell + 1))
                    opponent_mill_stones.add((ring, (cell + 2) % 8))
    for cell in [1, 3, 5, 7]:
        if(board[0][cell] == board[1][cell] == board[2][cell] == opponent_player):
            opponent_mill_stones.add((0, cell))
            opponent_mill_stones.add((1, cell))
            opponent_mill_stones.add((2, cell))
    
    opponent_stones = player_pieces(state, opponent_player)
    beatable_stones = [piece for piece in opponent_stones if piece not in opponent_mill_stones ]
    
    if len(beatable_stones) == 0: #falls kein anderer Stein geschlagen werden kann, dürfen auch Steine in Mühlen geschlagen werden
        return list(opponent_stones)
    return beatable_stones

`beat_stones()` gibt alle möglichen Spielfelder zurück, nachdem eine bestimmte Anzahl an gegnerischen Steinen (`count`) entfernt wurde

In [ ]:
def beat_stones(board, count, player):
    if count <= 0: return [board]
    
    boards = []
    beatable_stones = get_opponent_beatable_stones([[0, 0], board], player)
    for ring, cell in beatable_stones:
        new_board = copy.deepcopy(board)
        new_board[ring][cell] = 0
        boards.extend(beat_stones(new_board, count - 1, player))
    
    # Duplikate entfernen
    # TODO: eventuell geht das noch schöner
    result = []
    for board in boards:
        if board not in result:
            result.append(board)
    return result